In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self,dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx,dy

In [3]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

220.00000000000003


In [4]:
dprice = 1
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [5]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx,dy

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1


mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer  = MulLayer()

apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
all_price = add_apple_orange_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(all_price,tax)

dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice)
dapple_price,dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num  = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [8]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self,x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [9]:
x = np.array([[1.0,-0.5],[-2.0,3.0]])

In [10]:
x

array([[ 1. , -0.5],
       [-2. ,  3. ]])

In [11]:
mask = (x <= 0)

In [12]:
mask

array([[False,  True],
       [ True, False]])

In [13]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self,x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self,dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

In [14]:
x =np.random.rand(2)

In [15]:
x

array([0.52650739, 0.94827907])

In [16]:
w  = np.random.rand(2,3)

In [17]:
w

array([[0.51440575, 0.23976941, 0.90372193],
       [0.81731998, 0.21706992, 0.67748634]])

In [18]:
b = np.random.rand(3)

In [19]:
x.shape

(2,)

In [20]:
w.shape

(2, 3)

In [21]:
b.shape

(3,)

In [22]:
y = np.dot(x,w) + b

In [23]:
y

array([1.23497562, 0.43549082, 1.91902925])

In [24]:
x_dot_w = np.array([[0,0,0],[10,10,10]])

In [25]:
b = np.array([1,2,3])

In [26]:
x_dot_w

array([[ 0,  0,  0],
       [10, 10, 10]])

In [27]:
x_dot_w + b

array([[ 1,  2,  3],
       [11, 12, 13]])

In [28]:
dy = np.array([[1,2,3],[4,5,6]])

In [29]:
dy

array([[1, 2, 3],
       [4, 5, 6]])

In [30]:
db = np.sum(dy, axis=0)

In [31]:
db

array([5, 7, 9])

In [32]:
class Affine:
    def __init__(self,w,b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
    
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.w) + self.b
        
        return out
    
    def backward(self,dout):
        dx = np.dot(dout, self.w.t)
        self.dw = np.dot(self.x.t,dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [33]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) /batch_size
        
        return dx

In [36]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from layers import *
from gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['w1'] = weight_init_std * \
                                        np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['w2'] = weight_init_std * \
                                        np.random.randn(input_size,hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['w1'],self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['w2'],self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
            return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t)
        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_w = lambda w: self.loss(x.t)
        
        grads = {}
        grads['w1'] = numerical_gradient(loss_w,self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w,self.params['b1'])
        grads['w2'] = numerical_gradient(loss_w,self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w,self.params['b2'])
        
        return grads
    
    def gradient(self,x,t):
        self.loss(x,t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['w1'] = numerical_gradient(loss_w,self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w,self.params['b1'])
        grads['w2'] = numerical_gradient(loss_w,self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w,self.params['b2'])
        
        return grads
        

In [38]:
import sys,os
sys.path.append(os.pardir)
from mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test,t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch,t_batch)
gradbackprop = network.gradient(x_batch,t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(gradbackprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:2.0163917319939695e-10
b1:9.292530860873155e-10
W2:7.1726548230705e-08
b2:1.4366829812484382e-07


In [ ]:
import sys,os
sys.path.append(os.pardir)
from mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test,t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

iter_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learnig_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size,1)

for i in range(iter_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch,t_batch)
    
    for key in ('W1','b1','W2','b2'):
        network.params[key] -= learnig_rate * grad[key]
        
    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc,test_acc)

0.10218333333333333 0.101
0.7731833333333333 0.7771
0.8766333333333334 0.8799
0.8987666666666667 0.9032
0.9088333333333334 0.9122
0.9148833333333334 0.9178
0.9201166666666667 0.9208
